In [1]:
import pandas as pd
import os
import glob
import requests
from datetime import datetime, timedelta
from tkinter import *
import tkinter as tk
from tkinter import messagebox
import numpy as np
import time

from styleframe import StyleFrame, Styler, utils


In [2]:
import win32com.client
import subprocess
import shutil

In [3]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [4]:
Program_Start_Time  = time.time()
print("Current file location: "+ os.getcwd())
print("Your WBI is: "+ os.getlogin())

Current file location: c:\Python Script\Dummy PV Clean-up
Your WBI is: nxf83451


### SAP GUI

In [5]:
while True:
    try:
        SapGuiAuto = win32com.client.GetObject("SAPGUI") #Get currrent open SAP interface
        application = SapGuiAuto.GetscriptingEngine 
        connection = application.Children(0)
        session = connection.Children(0)
        break
    except:
        box = tk.Tk()
        box.withdraw()
        messagebox.showinfo('SAP GUI Notification','Please make sure activate your SAP GUI !!')


### Class and Function

In [6]:
class General_function:
     def __init__(self):
          self.output_folder = "./Input"

     def POP_message(self, input_string):
        box = tk.Tk()
        box.withdraw()
        messagebox.showinfo('Notification', input_string)
        pass
        
     def style_changes(self, input_frame, input_path, output_path, update_columns_combination, color_combination, sheet_name, pivot_frame):
          #Summary Table
          sf_output_ref = StyleFrame.read_excel(input_path, read_style= True)
          sf_output_ref.rename(columns= {"Unnamed: 0":"Index"}, inplace = True)
          
          for x in input_frame.columns:
               sf_output_ref[x] = input_frame[x].fillna("").values
               sf_output_ref[x] = sf_output_ref[x].astype(str)
               header_width = input_frame[x].astype(str).str.len().max()+5
               if header_width <= len(x):
                    header_width = len(x)+5

               sf_output_ref.apply_column_style(cols_to_style = x,
                                             styler_obj=Styler(font=utils.fonts.calibri,font_size= 11, border_type= "default_grid"),
                                             width=header_width,
                                   style_header=False)


          for col_list, color in zip(update_columns_combination, color_combination):
               sf_output_ref.apply_headers_style(styler_obj = Styler(font = 'Calibri', font_size = 11, bg_color = color), cols_to_style = col_list)
          
          #Pivot Table
          sf_pivot = StyleFrame(pivot_frame)
          for x in pivot_frame.columns:
               header_width = pivot_frame[x].astype(str).str.len().max()+5
               if header_width <= len(x):
                    header_width = len(x)+5

               sf_pivot.apply_column_style(cols_to_style = x,
                                           styler_obj=Styler(font=utils.fonts.calibri,font_size= 11, border_type= "default_grid"),
                                           width=header_width,
                                           style_header=False)
          sf_pivot.apply_headers_style(styler_obj = Styler(font = 'Calibri', font_size = 11, bg_color = "99FFFF"))

          #Export
          excel_writer = StyleFrame.ExcelWriter(output_path)
          sf_output_ref.to_excel(excel_writer, sheet_name = sheet_name, index= False, header = True)
          sf_pivot.to_excel(excel_writer, sheet_name='Pivot_table', index= False, header = True)
          excel_writer.save()

          pass

In [7]:
class SAP_Download:
    def __init__(self):        
        self.output_folder = os.getcwd() + "\\Input"
        self.today = (datetime.today()).strftime('%Y%m%d')
        self.input_ref = pd.read_excel("C:\Python Script\Dummy PV Clean-up\Input\Materials\Input_Table_name.xlsx")

    def Paste_data_to_clikpboard(self, action_selected):
        if action_selected == "ZPPBOMREP_Input":
            df_input_request = pd.read_excel("{}\\Input\\Export_MARA.xlsx".format(os.getcwd()))
            df_input_request["Material"].to_clipboard(excel=True, sep=None, index=False, header=None)
        elif action_selected == "MARC_Input":
            df_input_request = pd.read_excel("{}\\Input\\Export_ZPPBOMREP.xlsx".format(os.getcwd()))
            df_input_request["Component"] = df_input_request["Component"].astype("Int64")
            df_input_request["Component"].dropna().to_clipboard(excel=True, sep=None, index=False, header=None)
        elif action_selected == "T024_Input":
            df_input_request = pd.read_excel("{}\\Input\\Export_MARC.xlsx".format(os.getcwd()))
            df_input_request["PGr"].dropna().to_clipboard(excel=True, sep=None, index=False, header=None)
        elif action_selected == "MARA_Component_Input":
            df_input_request = pd.read_excel("{}\\Input\\Export_MARC.xlsx".format(os.getcwd()))
            df_input_request["Material"].to_clipboard(excel=True, sep=None, index=False, header=None)


    def MARA(self, download_type):
        try:
            start_time = time.time()
            session.findById("wnd[0]").maximize
            session.findById("wnd[0]/tbar[0]/okcd").text = "/nZSE16"
            session.findById("wnd[0]").sendVKey(0)
            session.findById("wnd[0]/usr/ctxtGV_TABNM").text = "MARA"
            session.findById("wnd[0]").sendVKey(0)
            if download_type == "MARA":
                session.findById("wnd[0]/usr/ctxtI1-LOW").text = "1*"
                layout_description = self.input_ref.loc[self.input_ref["Table_name"] == "MARA", "Layout_description"].item()
                output_file = self.input_ref.loc[self.input_ref["Table_name"] == "MARA", "Output_file_name"].item()
            elif download_type == "MARA_Component":
                session.findById("wnd[0]/usr/btn%_I1_%_APP_%-VALU_PUSH").press()
                self.Paste_data_to_clikpboard("MARA_Component_Input")
                session.findById("wnd[1]/tbar[0]/btn[16]").press()
                session.findById("wnd[1]").sendVKey(24)
                session.findById("wnd[1]").sendVKey(8)
                layout_description = self.input_ref.loc[self.input_ref["Table_name"] == "MARA_Component", "Layout_description"].item()
                output_file = self.input_ref.loc[self.input_ref["Table_name"] == "MARA_Component", "Output_file_name"].item()

            session.findById("wnd[0]/usr/txtMAX_SEL").text = ""
            session.findById("wnd[0]").sendVKey(8)
            session.findById("wnd[0]").sendVKey(33)
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").setCurrentCell(-1,"TEXT")
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectColumn("TEXT")
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectedRows = ""
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").contextMenu()
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectContextMenuItem("&FILTER")
            session.findById("wnd[2]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = layout_description
            session.findById("wnd[2]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").caretPosition = 17
            session.findById("wnd[2]").sendVKey(0)
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").currentCellColumn =("TEXT")
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectedRows = "0"
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").clickCurrentCell()
            session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell").setCurrentCell(-1,"MTART")
            session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell").selectColumn("MTART")
            session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell").contextMenu()
            session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell").selectContextMenuItem("&FILTER")
            session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = ""
            session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/btn%_%%DYN001_%_APP_%-VALU_PUSH").press()
            session.findById("wnd[2]/tbar[0]/btn[16]").press()
            if download_type == "MARA":
                session.findById("wnd[2]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = "PEP"
            session.findById("wnd[2]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").caretPosition = 3
            session.findById("wnd[2]").sendVKey(8)
            session.findById("wnd[1]").sendVKey(0)
            session.findById("wnd[0]").sendVKey(43)
            session.findById("wnd[1]").sendVKey(0)
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = os.getcwd() + "\\Input"
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = output_file
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 0
            session.findById("wnd[1]").sendVKey(11)

            status_msg = session.findById("wnd[0]/sbar").text
            print("{table_name} table --- finished downloading ! Time spent: {time_spent} mins, Status message: {status_msg}".format(table_name = download_type,
                                                                                                                                time_spent = round((time.time()-start_time)/60, 2),
                                                                                                                                status_msg = status_msg))
        except:
            print("{table_name} table --- Error with the table downloading".format(table_name = download_type))
        pass

    def MARC(self):
        try:
            start_time = time.time()
            session.findById("wnd[0]").maximize()
            session.findById("wnd[0]/tbar[0]/okcd").text = "/nZSE16"
            session.findById("wnd[0]").sendVKey(0)
            session.findById("wnd[0]/usr/ctxtGV_TABNM").text = "MARC"
            session.findById("wnd[0]/usr/ctxtGV_TABNM").caretPosition = 4
            session.findById("wnd[0]").sendVKey(0)
            session.findById("wnd[0]/usr/btn%_I1_%_APP_%-VALU_PUSH").press()
            session.findById("wnd[1]/tbar[0]/btn[16]").press()
            self.Paste_data_to_clikpboard("MARC_Input")
            session.findById("wnd[1]/tbar[0]/btn[24]").press()
            session.findById("wnd[1]").sendVKey(8)
            session.findById("wnd[0]/usr/txtMAX_SEL").text = ""
            session.findById("wnd[0]/usr/txtMAX_SEL").setFocus()
            session.findById("wnd[0]/usr/txtMAX_SEL").caretPosition = 11
            session.findById("wnd[0]").sendVKey(8)

            #Choose Layout
            session.findById("wnd[0]").sendVKey(33)
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").setCurrentCell(-1,"TEXT")
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectColumn("TEXT")
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").contextMenu()
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectContextMenuItem("&FILTER")
            session.findById("wnd[2]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = self.input_ref.loc[self.input_ref["Table_name"] == "MARC", "Layout_description"].item()
            session.findById("wnd[2]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").caretPosition = 17
            session.findById("wnd[2]").sendVKey(0)
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").currentCellColumn = "TEXT"
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectedRows = "0"
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").clickCurrentCell()
            session.findById("wnd[0]").sendVKey(43)
            session.findById("wnd[1]").sendVKey(0)
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = os.getcwd() + "\\Input"
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = self.input_ref.loc[self.input_ref["Table_name"] == "MARC", "Output_file_name"].item()
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 17
            session.findById("wnd[1]").sendVKey(11)

            status_msg = session.findById("wnd[0]/sbar").text
            print("{table_name} table --- finished downloading ! Time spent: {time_spent} mins, Status message: {status_msg}".format(table_name = "MARC",
                                                                                                                            time_spent = round((time.time()-start_time)/60, 2),
                                                                                                                            status_msg = status_msg))  

        except:
            print("{table_name} table --- Error with the table downloading".format(table_name = "MARC"))
        pass

    def ZPPBOMREP(self):
        try:
            start_time = time.time()
            session.findById("wnd[0]").maximize()
            session.findById("wnd[0]/tbar[0]/okcd").text = "/nZPPBOMREP"
            session.findById("wnd[0]").sendVKey(0)
            #Plant
            session.findById("wnd[0]/usr/ctxtSO_WERKS-LOW").text = "E0*"
            session.findById("wnd[0]/usr/ctxtSO_STLAN-LOW").text = "1"
            session.findById("wnd[0]/usr/ctxtSO_STLAN-LOW").setFocus()
            session.findById("wnd[0]/usr/ctxtSO_STLAN-LOW").caretPosition = 1
            session.findById("wnd[0]/usr/btn%_SO_WERKS_%_APP_%-VALU_PUSH").press()
            session.findById("wnd[1]/tbar[0]/btn[16]").press()
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = "E0*"
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "E1*"
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").text = "E2*"
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,3]").text = "E3*"
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").text = "E4*"
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,5]").text = "E5*"
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,5]").setFocus()
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,5]").caretPosition = 3
            session.findById("wnd[1]").sendVKey(8)

            #Material Number
            session.findById("wnd[0]/usr/btn%_SO_MATNR_%_APP_%-VALU_PUSH").press()
            session.findById("wnd[1]/tbar[0]/btn[16]").press()
            self.Paste_data_to_clikpboard("ZPPBOMREP_Input")
            session.findById("wnd[1]/tbar[0]/btn[24]").press()
            session.findById("wnd[1]").sendVKey(8)
            session.findById("wnd[0]/usr/ctxtPA_VARI").setFocus()
            session.findById("wnd[0]/usr/ctxtPA_VARI").caretPosition = 1
            session.findById("wnd[0]").sendVKey(8)
            session.findById("wnd[0]").sendVKey(33)
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").setCurrentCell(-1,"TEXT")
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectColumn("TEXT")
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectedRows = ""
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").contextMenu()
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectContextMenuItem("&FILTER")
            session.findById("wnd[2]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = "Dummy PV"
            session.findById("wnd[2]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").caretPosition = 8
            session.findById("wnd[2]").sendVKey(0)
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").currentCellColumn = "TEXT"
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectedRows = "0"
            session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").clickCurrentCell()
            session.findById("wnd[0]/mbar/menu[0]/menu[3]/menu[1]").select()
            session.findById("wnd[1]").sendVKey(0)
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = os.getcwd() + "\\Input"
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = self.input_ref.loc[self.input_ref["Table_name"] == "ZPPBOMREP", "Output_file_name"].item()
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 17
            session.findById("wnd[1]").sendVKey(11)
            
            status_msg = session.findById("wnd[0]/sbar").text
            print("{table_name} table --- finished downloading ! Time spent: {time_spent} mins, Status message: {status_msg}".format(table_name = "ZPPBOMREP",
                                                                                                                                time_spent = round((time.time()-start_time)/60, 2),
                                                                                                                                status_msg = status_msg))
        except:
            print("{table_name} table --- Error with the table downloading".format(table_name = "MARC"))  
        pass


    def T024(self):
        try:
            start_time = time.time()
            session.findById("wnd[0]").maximize()
            session.findById("wnd[0]/tbar[0]/okcd").text = "/nZSE16"
            session.findById("wnd[0]").sendVKey(0)
            session.findById("wnd[0]/usr/ctxtGV_TABNM").text = "T024"
            session.findById("wnd[0]").sendVKey(0)
            session.findById("wnd[0]/usr/btn%_I1_%_APP_%-VALU_PUSH").press()
            session.findById("wnd[1]/tbar[0]/btn[16]").press()
            self.Paste_data_to_clikpboard("T024_Input")
            session.findById("wnd[1]/tbar[0]/btn[24]").press()
            session.findById("wnd[1]").sendVKey(8)
            session.findById("wnd[0]/usr/txtMAX_SEL").text = ""
            session.findById("wnd[0]/usr/txtMAX_SEL").setFocus()
            session.findById("wnd[0]/usr/txtMAX_SEL").caretPosition = 10
            session.findById("wnd[0]").sendVKey(8)
            session.findById("wnd[0]/tbar[1]/btn[43]").press()
            session.findById("wnd[1]").sendVKey(0)
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = os.getcwd() + "\\Input"
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = self.input_ref.loc[self.input_ref["Table_name"] == "T024", "Output_file_name"].item()
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 16
            session.findById("wnd[1]").sendVKey(11)
                
            status_msg = session.findById("wnd[0]/sbar").text
            print("{table_name} table --- finished downloading ! Time spent: {time_spent} mins, Status message: {status_msg}".format(table_name = "T024",
                                                                                                                                time_spent = round((time.time()-start_time)/60, 2),
                                                                                                                                status_msg = status_msg))
                                                                                                                        
        except:
            print("{table_name} table --- Error with the table downloading".format(table_name = "T024"))  
        pass


In [8]:
import pandas as pd
import os
from datetime import datetime, timedelta

class Data_Processing:
    def __init__(self):
        self.today = (datetime.today()).strftime('%Y%m%d')
        self.input_path = os.getcwd() + "\\Input\\"
        self.output_folder = os.getcwd() + "\\Output\\"
        self.output_file_path = os.getcwd() + "\\Output\\DummyPV Clean-up_Final_{}.xlsx".format(self.today)
        self.output_msg = os.getcwd() + "\\Input\\Materials\\Email template.msg"

        self.input_PDQ213_path = os.getcwd() + "\\Input\\pdq213_PG_SUPPLY_CHAIN_with_Component_Info.csv"
        self.input_MARA_path = os.getcwd() + "\\Input\\Export_MARA.xlsx"
        self.input_ZPPBOMREP_path = os.getcwd() + "\\Input\\Export_ZPPBOMREP.xlsx"
        self.input_MARC_path = os.getcwd() + "\\Input\\Export_MARC.xlsx"
        self.input_T024_path = os.getcwd() + "\\Input\\Export_T024.xlsx"
        self.input_MARA_Component_path = os.getcwd() + "\\Input\\Export_MARA_Component.xlsx"


        self.ZPPBOMREP_column_list = ["Material", "Plant", "BOM Usage", "Production Version", "Alternative BOM", "Item Number", "Component"]
        self.Customized_column_list = ["MRP Type", "PDQ213", "Status", "Planner Code", "Planner Name", "Remarks"]
        self.PPF_feedback = ["PPF Feedback", "Reasons"]
    
    def main(self):
        start_time = time.time()

        df_phase_1 = self.Phase_1_ZPPBOMREP_and_PDQ()
        df_phase_2 = self.Phase_2_Final_Report_Generation(df_phase_1)
        df_pivot_result = self.Remarks_pivot(df_phase_2)
        combination_list = [self.ZPPBOMREP_column_list, self.Customized_column_list, self.PPF_feedback]
        color_list = ["#BFC9CA", "#99FFFF", "yellow"]
        General_function().style_changes(df_phase_2, 
                                         self.output_file_path, 
                                         self.output_file_path, 
                                         combination_list, color_list, "Summary", df_pivot_result)
        print("Analysis Process End ! Time spent: {time_spent} mins".format(time_spent = round((time.time()-start_time)/60, 2)))
        os.startfile(self.output_msg )
        os.startfile(self.output_folder)


    def Phase_1_ZPPBOMREP_and_PDQ(self):
        print("=========== Phase 1 Data Preprocessing --- ZPPBOMRPEP and PDQ213 Report Aggregation ===========")    

        #PDQ213
        # Get all PDQ ID with 27 characters and returb it as a list
        df_PDQ_ref = pd.read_csv(self.input_PDQ213_path)
        df_PDQ = pd.DataFrame([])
        df_PDQ["Produced"] = df_PDQ_ref["ITEM_NAME"].astype(str) + df_PDQ_ref["PRODUCED_LOCATION"].astype(str)
        df_PDQ["Consumed"] = df_PDQ_ref["CONSUMED_ITEM"].astype(str) + df_PDQ_ref["CONSUMED_LOCATION"].astype(str)
        Combination_Series = pd.concat([df_PDQ["Produced"], df_PDQ["Consumed"]], axis = 0).reset_index(drop=True)

        Combination_unique_list = []
        for x in Combination_Series:
            if len(x)==27 and x not in Combination_unique_list:
                Combination_unique_list.append(x)

        #ZPPBOMREP
        df_ZPPBOMREP = pd.read_excel(self.input_ZPPBOMREP_path)
        df_ZPPBOMREP = df_ZPPBOMREP.fillna("")

        df_ZPPBOMREP["Item Number"] = pd.Series(df_ZPPBOMREP["Item Number"], dtype="string").str.split(".", expand=True)[0]
        df_ZPPBOMREP["Item Number"] = df_ZPPBOMREP["Item Number"].apply(lambda x : ("0"*(4-len(x))) + x if len(x) < 4 and x != "" else x)

        #Convert the format of ID (Component + "00"+ 8 digits of material + @Plant)
        df_ZPPBOMREP.insert(loc=0, column='ID', value="")
        df_ZPPBOMREP["Material"] = pd.Series(df_ZPPBOMREP["Material"], dtype="string").str.split(".", expand=True)[0]
        df_ZPPBOMREP["Material_combination"] = df_ZPPBOMREP["Material"].apply(lambda x: "00" + x[:8] + "@")

        ID_list= ['Component', "Material_combination", 'Plant']
        for x in ID_list:
            df_ZPPBOMREP[x] = df_ZPPBOMREP[x].fillna("")
            df_ZPPBOMREP[x] = pd.Series(df_ZPPBOMREP[x], dtype="string").str.split(".", expand=True)[0]
            df_ZPPBOMREP["ID"] +=df_ZPPBOMREP[x]


        df_ZPPBOMREP["ID_LEN"] = df_ZPPBOMREP["ID"].apply(lambda x: len(x))
        df_ZPPBOMREP = df_ZPPBOMREP[df_ZPPBOMREP["ID_LEN"]==27]

        del df_ZPPBOMREP["Material_combination"]
        del df_ZPPBOMREP["ID_LEN"]


        #Reutrn and sort column order
        df = df_ZPPBOMREP.copy()
        df[self.Customized_column_list] = ""
        df[self.PPF_feedback] = ""
        df = df[[*["ID"], *self.Customized_column_list, *self.PPF_feedback, *self.ZPPBOMREP_column_list]]

        ID_intersection_set = set(df["ID"]).intersection(Combination_unique_list)
        df.loc[df["ID"].isin(ID_intersection_set), "PDQ213"] = df.loc[df["ID"].isin(ID_intersection_set), "ID"]
        df = df.fillna("")
        return df

    def Phase_2_Final_Report_Generation(self, input_phase_1_result):

        print("=========== Phase 2 Data Preprocessing --- MARC/T024/MARA Result Mapping ======================")    

        #MARC and T024
        df_final = input_phase_1_result.copy()
        df_final["Component@Plant"] = df_final["Component"].astype(str)+"@"+df_final["Plant"]
        df_MARC = pd.read_excel(self.input_MARC_path)
        df_MARC["Component@Plant"] =  df_MARC["Material"].astype(str) + "@" + df_MARC["Plnt"]
        df_T024 = pd.read_excel(self.input_T024_path)
        df_MARA_Component = pd.read_excel(self.input_MARA_Component_path)
        df_MARA_Component["Material"] = df_MARA_Component["Material"].astype(str)

        # Mapping dict
        df_MARC_MRPType_dict = dict(zip(df_MARC["Component@Plant"], df_MARC["Typ"]))
        df_MARC_PlannerCode_dict = dict(zip(df_MARC["Component@Plant"], df_MARC["PGr"]))
        df_TO24_Planner_dict = dict(zip(df_T024["PGr"], df_T024["Description"]))
        df_MARA_Status_dict = dict(zip(df_MARA_Component["Material"], df_MARA_Component["X-Plant Material Status"]))

        df_final["MRP Type"] = df_final["Component@Plant"].map(df_MARC_MRPType_dict)
        df_final["Planner Code"] = df_final["Component@Plant"].map(df_MARC_PlannerCode_dict)
        df_final["Planner Name"] = df_final["Planner Code"].map(df_TO24_Planner_dict)
        df_final["Status"] = df_final["Component"].map(df_MARA_Status_dict)

        # df_output["Item Number"] = pd.Series(df_output["Item Number"], dtype="string").str.split(".", expand=True)[0]
        # df_output["Item Number"] = df_output["Item Number"].apply(lambda x : ("0"*(4-len(x)))+x if len(x)<4 and x != "" else x)

        #%%
        df_final.loc[(df_final["PDQ213"]==""), "Remarks"] = "Not in PDQ213"
        df_final.loc[(df_final["Status"]=="OB") & (df_final["Remarks"]==""), "Remarks"] = "OBS Component"
        df_final["PDQ_Value_Counts"] = df_final["ID"].map(dict(df_final["ID"].value_counts()))
        df_final.loc[(df_final["PDQ_Value_Counts"] != 1) & (df_final["Remarks"]==""), "Remarks"] = "Duplicate Component"
        df_final.loc[(df_final["Remarks"]==""), "Remarks"] = "Exclude in Clean-up"

        #%%
        df_final = df_final.fillna("")
        df_final.to_excel(self.output_file_path, sheet_name="Summary", index= False)

        return df_final
    
    def Remarks_pivot(self, input_frame):
        summary_dict = input_frame.value_counts("Remarks")
        summary_table = pd.DataFrame([summary_dict.index, summary_dict.values]).T
        summary_table.columns = ["Remarks", "Count"]
        return summary_table

        



In [9]:
class Main:
    def __init__(self):
        self.today = (datetime.today()).strftime('%Y%m%d')
        self.PDQ_zip_path = r"C:\Users\{user}\NXP\S&P TPE - Merged_Reports\Merged_PDQ{date}\pdq213_PG_SUPPLY_CHAIN_with_Component_Info.zip".format(user = os.getlogin(),
                                                                                                                                                   date = (datetime.today()).strftime('%Y%m%d'))
        
    def Process(self):
        box = tk.Tk()
        box.withdraw()
        result = messagebox.askyesno('Confirmed Message', 'Check if you want to download all PDQ report and SAP tables automatically \n\n - Click "Yes" to trigger the auto-downloading process \n - Click "No" to skip the process')
        print(result)

        if result:
            print("=========== Step 1 --- Table Downloading ===========")
            #SAP action
            [os.remove(x) for x in glob.glob('./Input/*.xlsx')]
            self.PDQ_unpacked()
            SAP_Download().MARA("MARA")
            SAP_Download().ZPPBOMREP()
            SAP_Download().MARC()
            SAP_Download().T024()
            SAP_Download().MARA("MARA_Component")
            
            time.sleep(2)
            subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])
        else:
            os.startfile(os.getcwd() + "\\Input")
            messagebox.showwarning('Notification', "Check and ensure all reports are in the Input folder")
            time.sleep(1)

        #Data Preprocessing
        print("\n=========== Step 2 --- Final Repoert Generating ===========")
        Data_Processing().main()
        

    def PDQ_unpacked(self):
        src = self.PDQ_zip_path
        dst = os.getcwd() +"\\"+ "Input"
        zip_file = dst + "\\pdq213_PG_SUPPLY_CHAIN_with_Component_Info.zip"
        shutil.copy(src, dst)                                                                                                                
        shutil.unpack_archive(filename = zip_file, 
                              extract_dir = dst,
                              format = "zip")
        os.remove(zip_file)



In [10]:
Main().Process()

False

=========== Step 2 --- Final Repoert Generating ===========
=========== Phase 1 Data Preprocessing --- ZPPBOMRPEP and PDQ213 Report Aggregation ===========
=========== Phase 2 Data Preprocessing --- MARC/T024/MARA Result Mapping ======================
Analysis Process End ! Time spent: 0.68 mins


### Completion message

In [11]:
box = tk.Tk()
box.withdraw()
messagebox.showinfo('Notification', "Total Process Time: {} mins !".format(round((time.time()-Program_Start_Time)/60, 2)))

'ok'

In [12]:
# import tkinter as tk
# from tkinter import messagebox

# box = tk.Tk()
# box.withdraw()
# result = messagebox.askyesno('Confirmed Message', 'Please check')
# print(result)
